In [41]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from skimage.filters import threshold_otsu, sobel
from skimage.segmentation import flood, flood_fill
from glob import glob

In [42]:
fileList = glob('*.jpg')
images = []
for i in range(5):   
    image = io.imread("Batch 3-4 Co -20%/Sample 3B topcoat 0{:d} BSD.jpg".format(i + 1))
    images.append(image)

In [43]:
def process(images):
    row_of_black = np.zeros(images[0].shape[1])
    row_of_gray = np.full(shape=images[0].shape[1], fill_value=170, dtype=np.int)
    processed = []
    for image in images:
        image = image[:1760, :]
        image = np.vstack([row_of_black, image])
        image = np.vstack([image, row_of_gray])
        image = flood_fill(image, (0,0), 255, tolerance=30)
        image = flood_fill(image, (image.shape[0]-1, 0), 255, tolerance=40)

        processed.append(image)
    
    return processed

In [44]:
processed = process(images)

In [45]:
def find_ysz_layer(images):
    seed = (0,0)
    
    ysz_layers = []
    for image in images:
        img_sobel = sobel(image)
        mask = flood(img_sobel, seed, tolerance = 0.01)
        ysz_layers.append(mask)
    
    return ysz_layers

In [46]:
ysz_layers = find_ysz_layer(processed)

In [47]:
def apply_threshold(images, ysz_layers):
    thresh_images = []
    
    for image, layer in zip(images, ysz_layers):
        threshold = threshold_otsu(image)
        thresh_image = []
        
        for pixel, masked_pixel in zip(image.flatten(), layer.flatten()):
            if masked_pixel == False:
                pixel = pixel > threshold
                thresh_image.append(pixel)
                
        thresh_images.append(thresh_image)
    
    return thresh_images

In [48]:
thresh_images = apply_threshold(processed, ysz_layers)

In [49]:
def find_porosity(layers):
    porosities = []
    for layer in layers:
        porosity = 100 * ((len(layer) - np.sum(layer)) / len(layer))
        porosities.append(round(porosity, 2))
    
    return porosities

In [50]:
porosities = find_porosity(thresh_images)
print(porosities)

[13.86, 12.76, 12.53, 11.85, 11.26]
